In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np

import chess
import chess.pgn
from stockfish import Stockfish
import os

import uuid        # for id generation
import shortuuid

%load_ext autoreload
%autoreload 2

In [3]:
#stockfish = Stockfish(
#    'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#    parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
#)
#stockfish.set_elo_rating(2600)
#stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'FEN_moves': [],
    'Bitmap_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'turn': [],
    'Castling_right': [],
    'EP_option': [],
    'Pseudo_EP_option': [],
    'Halfmove_clock': []
    #'Result': [],
}

# Set list of Pieces
PIECES = [chess.Piece.from_symbol('P'),
         chess.Piece.from_symbol('N'),
         chess.Piece.from_symbol('B'),
         chess.Piece.from_symbol('R'),
         chess.Piece.from_symbol('Q'),
         chess.Piece.from_symbol('K'),
         chess.Piece.from_symbol('p'),
         chess.Piece.from_symbol('n'),
         chess.Piece.from_symbol('b'),
         chess.Piece.from_symbol('r'),
         chess.Piece.from_symbol('q'),
         chess.Piece.from_symbol('k')]

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # MOVE CYCLE
        white = True
        for move in moves:
            board.push(move)
            #fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
            #cpl = stockfish.get_evaluation()['value']/100
            
            moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
            moves_log_dict['FEN_moves'].append(board.fen())
            
            #Generate bitmap representation of FENs
            bitmap_board_dict = {}
            positions = board.piece_map()

            for piece in PIECES:
                bitmap_board = {}
                for position in positions:
                    if positions[position] == piece: 
                        bitmap_board[position] = 1
                    else:
                        bitmap_board[position] = 0
                bitmap_board_dict[str(piece)] = bitmap_board            
            
            moves_log_dict['Bitmap_moves'].append(bitmap_board_dict)
            
            #moves_log_dict['cpl'].append(cpl)
            
            #Turn color and castling availablity
            moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
            if white:
                moves_log_dict['turn'].append('white')
                moves_log_dict['Castling_right'].append(int(board.has_castling_rights(chess.WHITE)))
                white = False
            else:
                moves_log_dict['turn'].append('black')
                moves_log_dict['Castling_right'].append(int(board.has_castling_rights(chess.BLACK)))
                white = True
                
            #(Pseudo) en passant opportunity
            moves_log_dict['EP_option'].append(int(board.has_legal_en_passant()))
            moves_log_dict['Pseudo_EP_option'].append(int(board.has_pseudo_legal_en_passant()))
            
            #Halfmove clock
            moves_log_dict['Halfmove_clock'].append(board.halfmove_clock)
                
        game_counter += 1
        if game_counter == 50:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

50 games read.
CPU times: user 869 ms, sys: 5.09 ms, total: 874 ms
Wall time: 873 ms


## Players

In [5]:
df_players = pd.DataFrame(players)
#TODO player ID
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [6]:
black = list(df_players["Black"]) 
white = list(df_players["White"])
    
#merge uniqe values from both columns:
bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])

In [7]:
def finding_comp():
    comp1 = df_players.loc[df_players['WhiteIsComp'] == "Yes"]
    comp1 = comp1[['White']].copy()
    comp1.drop_duplicates(inplace=True)
    comp2 = df_players.loc[df_players['WhiteIsComp'] == "No"]
    comp2 = comp2[['Black']].copy()
    comp2.drop_duplicates(inplace=True)
    
    c1 = list(comp1['White'])
    c2 = list(comp2['Black'])
    computer = list(set(c1 + c2))
    
    computer = pd.DataFrame({'Computer' : computer})
    computer['Yes'] = 'Yes'
    
    
    return computer

In [8]:
finding_comp()

,Computer,Yes
0,chesspickle,Yes
1,nakshatra,Yes
2,GriffySr,Yes
3,scalaQueen,Yes
4,foggydew,Yes
5,Notarious,Yes
6,exeComp,Yes
7,IFDThor,Yes
8,forlat,Yes


In [9]:
#generates unique IDs from int
def id_generator(id):
    return uuid.uuid4().int

In [10]:
#generates unique IDs containing digits and characters
def short_id_gen(id):
    return shortuuid.ShortUUID().random(length=15)

In [11]:
def game_id(input_df):
    '''generates IDs for df_games'''
    input_df['Game_ID'] = input_df['Game_ID'].apply(short_id_gen)
    df_games = input_df
    return df_games

In [12]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids
    
    computer = finding_comp()
    players_id = players_id.merge(computer, left_on='Players', right_on='Computer', how='outer')
    players_id = players_id.replace(np.nan, 'No')
    players_id.drop(columns='Computer', inplace=True)
    players_id.rename(columns={'Yes' : 'Computer'}, inplace=True)
    
    return players_id

In [13]:
players_id = players_id_list(df_players, players_id)
players_id

,Player_ID,Players,Computer
0,66375623938346995765036435984668045422,Arsyah,No
1,164283175980360330640905005296537021265,scalaQueen,Yes
2,43919303604873068186162982430027363273,Ghannoum,No
3,12130288161796172035422018477023144824,Notarious,Yes
4,68446788283811137660518229256919072567,forlat,Yes
5,230657688777912479639236000507151389220,chesspickle,Yes
6,67303025062230105366619192785630117448,nakshatra,Yes
7,267980779800240896779249066492753290195,Bjboy,No
8,162426962758308286631087088644450814578,sparse,No
9,247603511474220471620049075423615631304,JMM,No


In [14]:
# dummy_games = df_games
# dummy_games.iloc[1] = ['jePsyVtxwueeBoR', '2021.01.31', 'Bambi', '1109', 'forlat', '2204', 'C44', '0-1']

In [15]:
# dummy_games.iloc[-1] = ['jePsyVtxwueeBoR', '2021.01.31', 'Geforce', '2207', 'forlat', '2204', 'C44', '0-1']

In [16]:
def find_and_replace_duplicates(df, subset):
    while True:
        if df.duplicated(subset=subset, keep=False).any() == True:
            is_dup = df.duplicated(subset=subset, keep="first")
            if subset == 'Game_ID':
                df[subset] = df[subset].where(~is_dup, df[subset].apply(short_id_gen))
            return df
            df[subset] = df[subset].where(~is_dup, df[subset].apply(id_generator))
        else: print(f'\u2713 no duplicates in {subset}s \u2713')
        break

In [17]:
find_and_replace_duplicates(players_id, "Player_ID")

✓ no duplicates in Player_IDs ✓


In [18]:
# players_id = players_id.merge(computer, left_on='Players', right_on='Computer', how='outer')
# players_id

In [19]:
# players_id.drop(columns='Computer', inplace=True)

In [20]:
# players_id.rename(columns={'Yes': 'Computer'})
# players_id.replace(np.nan, 'No')

In [21]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [22]:
players_id.dtypes == object

Player_ID    True
Players      True
Computer     True
dtype: bool

In [23]:
#players_id = players_id_list(df_dummy, players_id)

In [24]:
players_id.shape[0] == len(players_id["Player_ID"].unique())

True

In [25]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players']).astype(object)
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID", 'Computer_x', 'Computer_y'], inplace=True)
    df_players = m_bw
    return df_players

In [26]:
assign_player_id(df_players)

,White,White_Elo,Black,Black_Elo,WhiteIsComp,White_ID,Black_ID
0,forlat,1970,Geforce,2204,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
1,forlat,1976,Geforce,2198,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
2,forlat,1958,Geforce,2216,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
3,forlat,1964,Geforce,2210,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
4,forlat,1970,Geforce,2204,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
5,forlat,1976,Geforce,2198,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
6,forlat,1976,Geforce,2194,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
7,forlat,1984,Geforce,2186,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
8,forlat,1976,Geforce,2186,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256
9,forlat,1984,Geforce,2178,Yes,68446788283811137660518229256919072567,2367666463457784354790399060310841256


In [27]:
# def assign_player(input_df): # returns a df with 2 new columns and assigned player ID
#     #print("List od IDs has been generated:")
#     #print(players_unique)
#     m_white = pd.concat(input_df, players_id, join = 'outer')   #
#     m_white['White_ID'] = m_white['Player_ID']
#     m_white.drop(columns=['Players', "Player_ID"], inplace=True)
#     m_bw = m_white.concat(players_id, left_on=["Black"], right_on=['Players'])
#     m_bw['Black_ID'] = m_bw['Player_ID']
#     m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
#     df_players = m_bw
#     return df_players

In [28]:
#set.union(set(df_players['White']),(set(df_players['Black'])))

## Games

In [29]:
# def player_ids_to_games(df_games):   
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
#     return df_games

In [30]:
df_games = pd.DataFrame(games)
#TODO date to datetime

# def game_id(input_df):
#     '''generates IDs for df_games'''
#     input_df["old_ID"] = input_df["Game_ID"]
#     input_df['Game_ID'] = input_df['Game_ID'].apply(short_id_gen)
#     return df_games

def game_id():
    df_games["old_ID"] = df_games["Game_ID"]
    df_games["Game_ID"] = df_games['Game_ID'].apply(short_id_gen)
    
    return df_games




In [31]:
game_id()

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result,old_ID
0,3VL6xWihgesCHZT,2021.01.31,forlat,1970,Geforce,2204,A01,0-1,490373548
1,5GmRAdfqj2yU4na,2021.01.31,Geforce,2201,forlat,1973,B00,1-0,490373541
2,AfWXmxWcvkTaQQZ,2021.01.31,forlat,1976,Geforce,2198,A20,0-1,490373526
3,9ZF8EPbLUuJqHit,2021.01.31,Geforce,2211,forlat,1963,B10,0-1,490373517
4,8oTy9dqH2LqpE8F,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2,490373487
5,3vkih44yicotB6T,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0,490373455
6,37DAoQFXJnDomr3,2021.01.31,Geforce,2213,forlat,1961,B00,1-0,490373444
7,RWorfB7DSmm59B8,2021.01.31,forlat,1964,Geforce,2210,C28,0-1,490373432
8,9ATPWZ9QzD6Mhcz,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1,490373410
9,3rYzKYKxZyXB8pY,2021.01.31,Geforce,2207,forlat,1967,B10,1-0,490373400


In [32]:
#     data = [df_games["Game_ID"], df_games["old_ID"]]
#     headers = ["Game_ID", "old_ID"]
#     df3 = pd.concat(data, axis=1, keys=headers)
#     return df3

# def game_ids(df_games):
#     old_and_new_ids()
    
#     # merge df_games with players_id
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
    
#     return df_games

# def merging_b_w_ids(df_games):
#     '''returns a df_games with additional White_ID, Black_ID'''
#     df_games["old_ID"] = df_games["Game_ID"]
#     df_games["Game_ID"] = game_id(df_games)
    
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
#     return df_games

In [33]:
games_dict = df_games.to_dict()

In [34]:
'Game_ID' and 'old_ID' in games_dict.keys()

True

In [35]:
games_dict.keys()

dict_keys(['Game_ID', 'Date', 'White', 'White_Elo', 'Black', 'Black_Elo', 'ECO', 'Result', 'old_ID'])

In [36]:
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result,old_ID
0,3VL6xWihgesCHZT,2021.01.31,forlat,1970,Geforce,2204,A01,0-1,490373548
1,5GmRAdfqj2yU4na,2021.01.31,Geforce,2201,forlat,1973,B00,1-0,490373541
2,AfWXmxWcvkTaQQZ,2021.01.31,forlat,1976,Geforce,2198,A20,0-1,490373526
3,9ZF8EPbLUuJqHit,2021.01.31,Geforce,2211,forlat,1963,B10,0-1,490373517
4,8oTy9dqH2LqpE8F,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2,490373487
5,3vkih44yicotB6T,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0,490373455
6,37DAoQFXJnDomr3,2021.01.31,Geforce,2213,forlat,1961,B00,1-0,490373444
7,RWorfB7DSmm59B8,2021.01.31,forlat,1964,Geforce,2210,C28,0-1,490373432
8,9ATPWZ9QzD6Mhcz,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1,490373410
9,3rYzKYKxZyXB8pY,2021.01.31,Geforce,2207,forlat,1967,B10,1-0,490373400


## Moves

In [37]:
df_moves = pd.DataFrame(moves_log_dict)

In [38]:
def move_id(df_moves, df_games):  
    data = [df_games["Game_ID"], df_games["old_ID"]]
    headers = ["Game_ID", "old_ID"]
    df = pd.concat(data, axis=1, keys=headers)
    merging = df_moves.merge(df, how="left", left_on="Game_ID", right_on="old_ID")
    merging.drop(columns=["Game_ID_x", "old_ID"], inplace=True)
    merging.rename(columns = {'Game_ID_y' : 'Game_ID'}, inplace = True)
    merging.insert(0, "Moves_ID", merging.apply(lambda row: f"{row.Game_ID }-{row.turn}-{row.Halfmove_clock}", axis=1))
    df_moves = merging
    return df_moves

In [39]:
move_id(df_moves, df_games)

,Moves_ID,FEN_moves,Bitmap_moves,WhiteIsComp,turn,Castling_right,EP_option,Pseudo_EP_option,Halfmove_clock,Game_ID
0,3VL6xWihgesCHZT-white-0,rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR ...,"{'P': {63: 0, 62: 0, 61: 0, 60: 0, 59: 0, 58: ...",Yes,white,1,0,0,0,3VL6xWihgesCHZT
1,3VL6xWihgesCHZT-black-0,rnbqkbnr/pppp1ppp/8/4p3/8/1P6/P1PPPPPP/RNBQKBN...,"{'P': {63: 0, 62: 0, 61: 0, 60: 0, 59: 0, 58: ...",Yes,black,1,0,0,0,3VL6xWihgesCHZT
2,3VL6xWihgesCHZT-white-1,rnbqkbnr/pppp1ppp/8/4p3/8/1PN5/P1PPPPPP/R1BQKB...,"{'P': {63: 0, 62: 0, 61: 0, 60: 0, 59: 0, 58: ...",Yes,white,1,0,0,1,3VL6xWihgesCHZT
3,3VL6xWihgesCHZT-black-2,r1bqkbnr/pppp1ppp/2n5/4p3/8/1PN5/P1PPPPPP/R1BQ...,"{'P': {63: 0, 62: 0, 61: 0, 60: 0, 59: 0, 58: ...",Yes,black,1,0,0,2,3VL6xWihgesCHZT
4,3VL6xWihgesCHZT-white-0,r1bqkbnr/pppp1ppp/2n5/4p3/8/1PN1P3/P1PP1PPP/R1...,"{'P': {63: 0, 62: 0, 61: 0, 60: 0, 59: 0, 58: ...",Yes,white,1,0,0,0,3VL6xWihgesCHZT
...,...,...,...,...,...,...,...,...,...,...
2672,8kAsEi2734KYBCd-white-1,8/5pk1/4p2p/6p1/p2K2PP/3B2P1/1r6/8 b - - 1 41,"{'P': {54: 0, 53: 0, 47: 0, 44: 0, 38: 0, 31: ...",Yes,white,0,0,0,1,8kAsEi2734KYBCd
2673,8kAsEi2734KYBCd-black-0,8/5pk1/4p2p/6p1/3K2PP/p2B2P1/1r6/8 w - - 0 42,"{'P': {54: 0, 53: 0, 47: 0, 44: 0, 38: 0, 31: ...",Yes,black,0,0,0,0,8kAsEi2734KYBCd
2674,8kAsEi2734KYBCd-white-1,8/5pk1/4p2p/6p1/6PP/p1KB2P1/1r6/8 b - - 1 42,"{'P': {54: 0, 53: 0, 47: 0, 44: 0, 38: 0, 31: ...",Yes,white,0,0,0,1,8kAsEi2734KYBCd
2675,8kAsEi2734KYBCd-black-2,8/5pk1/1r2p2p/6p1/6PP/p1KB2P1/8/8 w - - 2 43,"{'P': {54: 0, 53: 0, 47: 0, 44: 0, 41: 0, 38: ...",Yes,black,0,0,0,2,8kAsEi2734KYBCd


In [40]:
pgn2 = open("../raw_data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player

game_string_list = []
game_counter = 0

while True: 
    
    game = chess.pgn.read_game(pgn2)
    game_string = str(game)
    game_string_list.append(game_string)
    
    game_counter += 1
    if game_counter == 50:  # number of games to read        
        break

print(game_string_list[15])

[Event "FICS rated blitz game"]
[Site "FICS freechess.org"]
[Date "2021.01.31"]
[Round "?"]
[White "Geforce"]
[Black "forlat"]
[Result "1-0"]
[BlackClock "0:03:00.000"]
[BlackElo "1986"]
[BlackIsComp "Yes"]
[BlackRD "0.0"]
[ECO "B00"]
[FICSGamesDBGameNo "490373183"]
[PlyCount "41"]
[Time "22:38:00"]
[TimeControl "180+0"]
[WhiteClock "0:03:00.000"]
[WhiteElo "2194"]
[WhiteRD "0.0"]

1. e4 { [%emt 0.0] } 1... Nc6 { [%emt 0.0] } 2. Nf3 { [%emt 0.581] } 2... e6 { [%emt 0.214] } 3. Nc3 { [%emt 0.561] } 3... Be7 { [%emt 1.164] } 4. d4 { [%emt 2.316] } 4... b6 { [%emt 1.506] } 5. Bd3 { [%emt 0.643] } 5... Nh6 { [%emt 0.951] } 6. a3 { [%emt 1.101] } 6... O-O { [%emt 0.914] } 7. h4 { [%emt 1.955] } 7... Bf6 { [%emt 1.153] } 8. e5 { [%emt 0.306] } 8... Be7 { [%emt 1.599] } 9. Bxh6 { [%emt 0.4] } 9... gxh6 { [%emt 1.764] } 10. Qd2 { [%emt 0.1] } 10... Bb7 { [%emt 1.391] } 11. Qxh6 { [%emt 0.831] } 11... f5 { [%emt 1.205] } 12. exf6 { [%emt 1.531] } 12... Rf7 { [%emt 1.797] } 13. Ng5 { [%emt 0.382

## Tests to check

In [ ]:
# def test_players_id_list():
#     '''checks if the length of df equals the number of unique IDs '''
#     df_players, df_games, df_moves = ChessData().import_data()
#     players_id = pd.DataFrame({'Players': [], 'Player_ID' : [], 'Computer' : []})
#     players_id_list(df_players)
#     assert df_players.shape[0] == len(df_players["Player_ID"].unique())
#     #assert players_id.isnull().values.any() == 0

# def test_player_id():
#     df_players, df_games, df_moves = ChessData().import_data()
#     new_df = player_id(df_players)
#     new_df_dic = new_df.to_dict()
#     assert df_players.isnull().values.any() == 0
#     assert df_players != new_df
#     assert 'White_ID' and 'Black_ID' in new_df_dic.keys()

# def test_game_id():
#     '''checks if the shape of new df'''
#     df_players, df_games, df_moves = ChessData().import_data()
#     games = game_id(df_games)
#     #games_dict = games.to_dict()
#     assert df_games.shape[1] != games.shape[1]
#     #assert 'Game_ID' and 'old_ID' in games_dict.keys()

# def test_move_id():
#     #Game_ID value is in Move_ID
#     data = [df_games["Game_ID"], df_games["old_ID"]]
#     headers = ["Game_ID", "old_ID"]
#     df = pd.concat(data, axis=1, keys=headers)

In [41]:
game.headers["Time"]

'14:24:00'

# search

In [42]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')

In [43]:
search_df(df_moves, 'ECO', 'A01')

'ECO' not found.


In [44]:
move1 = chess.Move.from_uci(uci="g8f6")
move1

Move.from_uci('g8f6')

In [45]:
# move = df_moves["FEN_moves"]
# board = chess.Board(move)
# board

In [46]:
#board.lan(move=move1)

In [47]:
positions = board.piece_map()

In [48]:
PIECES = list(set((val) for val in positions.values()))
PIECES

[Piece.from_symbol('P'),
 Piece.from_symbol('B'),
 Piece.from_symbol('K'),
 Piece.from_symbol('p'),
 Piece.from_symbol('r'),
 Piece.from_symbol('k')]

In [49]:
SQUARES = [i for i in range(64)]

In [50]:
for square in SQUARES:
    print(square)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [51]:
54 in positions.keys()

True

In [52]:
bitmap_board_dict = {}

for piece in PIECES:
    bitmap_board = {}
    for square in SQUARES:
        if square in positions.keys():
            if positions[square] == piece: 
                bitmap_board[square] = 1
            else:
                bitmap_board[square] = 0
        else:
            bitmap_board[square] = 0
    bitmap_board_dict[str(piece)] = bitmap_board

In [53]:
bitmap_board_dict

{'P': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
  15: 0,
  16: 0,
  17: 0,
  18: 0,
  19: 0,
  20: 0,
  21: 0,
  22: 1,
  23: 0,
  24: 0,
  25: 0,
  26: 0,
  27: 0,
  28: 0,
  29: 0,
  30: 1,
  31: 1,
  32: 0,
  33: 0,
  34: 0,
  35: 0,
  36: 0,
  37: 0,
  38: 0,
  39: 0,
  40: 0,
  41: 0,
  42: 0,
  43: 0,
  44: 0,
  45: 0,
  46: 0,
  47: 0,
  48: 0,
  49: 0,
  50: 0,
  51: 0,
  52: 0,
  53: 0,
  54: 0,
  55: 0,
  56: 0,
  57: 0,
  58: 0,
  59: 0,
  60: 0,
  61: 0,
  62: 0,
  63: 0},
 'B': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
  15: 0,
  16: 0,
  17: 0,
  18: 0,
  19: 0,
  20: 0,
  21: 0,
  22: 0,
  23: 0,
  24: 0,
  25: 0,
  26: 1,
  27: 0,
  28: 0,
  29: 0,
  30: 0,
  31: 0,
  32: 0,
  33: 0,
  34: 0,
  35: 0,
  36: 0,
  37: 0,
  38: 0,
  39: 0,
  40: 0,
  41: 0,
  42: 0,
  43: 0,
  44: 0,
  45: 0,
  46: 0,
  47: 0,
 

In [54]:
board.has_legal_en_passant()

False

In [55]:
board.has_pseudo_legal_en_passant()

False

In [56]:
board.halfmove_clock

3

# Data import with package

In [57]:
player_df2, game_df2, move_df2 = data.ChessData().import_data(data_path='../raw_data/Fics_data_pc_data.pgn', 
                                                              import_lim=50)


NameError: name 'data' is not defined

In [ ]:
move_df2

## Binary vector representation for board

In [ ]:
df_test = pd.DataFrame(move_df2['Bitmap_moves'][0])

In [ ]:
df_test

In [ ]:
dict_wide = {}

for index1, i in enumerate(df_test.columns):
    for index2, j in enumerate(df_test.index):
        dict_wide[str(i)+str(j)] = [df_test.iloc[index2, index1]]

In [ ]:
dict_wide

In [ ]:
pd.DataFrame(dict_wide)

In [ ]:
from cc_detector.move import binary_board_df

In [ ]:
df_wide = binary_board_df(move_df2)
#very inefficient so far (takes very long, probably needs to be redone differently)

In [ ]:
df_wide